# Analyse des habitudes d'achat Steam avec l'algorithme Apriori

Ce notebook accompagne un projet d'analyse de données Steam, utilisant l'algorithme Apriori pour extraire des règles d'association et générer des recommandations de jeux. Nous passerons par les étapes suivantes :

1. **Importation des bibliothèques**
2. **Téléchargement et chargement des données depuis KaggleHub**
3. **Prétraitement et nettoyage**
4. **Construction des transactions**
5. **Extraction des itemsets fréquents et génération des règles**
6. **Génération de règles au format CLIPS**
7. **Test interactif des règles CLIPS**


In [ ]:
# %%
# 1. Import des bibliothèques
import os
import re
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import kagglehub  # interface personnalisée pour Kaggle
import clips      # API Python pour CLIPS

---

## 2. Télécharger et charger les données Steam

Dans cette section, nous téléchargeons le jeu de données « @tamber/steam-video-games » depuis KaggleHub et le chargeons dans un DataFrame pandas. Nous vérifions également l'intégrité du fichier.

In [ ]:
# %%
def load_steam_data(dataset_name: str = "tamber/steam-video-games",
                    file_name: str = "steam-200k.csv") -> pd.DataFrame:
    # Téléchargement via KaggleHub
    path = kagglehub.dataset_download(dataset_name)
    print(f"Dataset téléchargé dans : {path}")

    # Construction du chemin vers le fichier et vérification
    file_path = os.path.join(path, file_name)
    if not os.path.exists(file_path) or os.path.getsize(file_path) < 1000:
        raise FileNotFoundError(f"Fichier introuvable ou corrompu : {file_path}")

    # Lecture du CSV sans en-tête, avec noms de colonnes personnalisés
    col_names = ['user_id', 'game_name', 'purchase', 'playtime', 'unused']
    df = pd.read_csv(file_path,
                     encoding='latin-1',
                     delimiter=',',
                     header=None,
                     names=col_names)

    print("Aperçu des données Steam (5 premières lignes) :")
    display(df.head())
    return df

# Chargement des données
steam_df = load_steam_data()
print(f"Nombre total de lignes initiales : {len(steam_df)}")

---

## 3. Prétraitement et nettoyage des données

Nous conservons uniquement les colonnes `user_id` et `game_name`, supprimons les doublons et retirons la colonne inutilisée.

In [ ]:
# %%
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    # On garde uniquement les colonnes d'intérêt et on supprime les doublons
    df = df[['user_id', 'game_name']]
    df = df.drop_duplicates()
    return df

# Application du prétraitement
clean_df = preprocess_data(steam_df)
print(f"Nombre de lignes après nettoyage : {len(clean_df)}")
display(clean_df.head())

---

## 4. Construction des transactions pour l'algorithme Apriori

Nous groupons les jeux par utilisateur pour obtenir une liste de transactions, où chaque transaction représente l'ensemble des jeux d'un utilisateur.

In [ ]:
# %%
def build_transactions(df: pd.DataFrame) -> list:
    grouped = df.groupby('user_id')['game_name'].apply(list)
    return grouped.tolist()

transactions = build_transactions(clean_df)
print(f"Nombre de transactions : {len(transactions)}")

---

## 5. Extraction des itemsets fréquents et génération des règles d'association

Nous convertissons les transactions en matrice binaire, appliquons l'algorithme Apriori pour extraire les itemsets fréquents, puis générons les règles d'association avec un seuil de confiance minimal.

In [ ]:
# %%
def mine_association_rules(transactions: list,
                           min_support: float = 0.02,
                           min_confidence: float = 0.5) -> pd.DataFrame:
    # Encodage des transactions au format binaire
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    trans_df = pd.DataFrame(te_ary, columns=te.columns_)

    # Extraction des itemsets fréquents
    frequent_itemsets = apriori(trans_df,
                                min_support=min_support,
                                use_colnames=True)

    # Génération des règles d'association
    rules = association_rules(frequent_itemsets,
                               metric="confidence",
                               min_threshold=min_confidence)
    return rules

# Extraction des règles avec support >= 2% et confiance >= 60%
rules = mine_association_rules(transactions,
                               min_support=0.02,
                               min_confidence=0.6)
print(f"Nombre de règles générées : {len(rules)}")
display(rules.head())

---

## 6. Génération des règles au format CLIPS

Pour utiliser CLIPS, nous convertissons les noms de jeux en symboles compatibles et écrivons les règles simples (1 antécédent → 1 conséquent) dans un fichier `.clp`. Nous pouvons également ajouter des faits de test.

In [ ]:
# %%
def sanitize(name: str) -> str:
    # Remplace les caractères spéciaux par des underscores
    name = re.sub(r"[&()/\-]", " ", name)
    name = re.sub(r"[^a-zA-Z0-9]", "_", name)
    name = re.sub(r"_+", "_", name).strip("_")
    return name


def generate_clips_rules(rules: pd.DataFrame,
                          output_file: str = "regles_steam.clp",
                          max_antecedents: int = 1,
                          max_consequents: int = 1,
                          test_items: list = None) -> None:
    if test_items is None:
        test_items = []

    with open(output_file, "w", encoding="utf-8") as f:
        for i, row in rules.iterrows():
            antecedents = list(row['antecedents'])
            consequents = list(row['consequents'])

            # Filtre pour règles simples
            if len(antecedents) != max_antecedents or len(consequents) != max_consequents:
                continue

            ant = sanitize(antecedents[0])
            cons = sanitize(consequents[0])
            f.write(f"(defrule regle-{i+1}\n")
            f.write(f"  (achat {ant})\n")
            f.write("  =>\n")
            f.write(f"  (assert (achat {cons}))\n")
            f.write(f"  (printout t \"Regle activée : {ant} => {cons}\" crlf))\n\n")

        # Ajout des cas de test si fournis
        if test_items:
            f.write("(deffacts cas-test\n")
            for item in test_items:
                f.write(f"  (achat {sanitize(item)})\n")
            f.write(")\n")

    print(f"Fichier CLIPS généré : {output_file}")

# Génération du fichier de règles
generate_clips_rules(rules)

---

## 7. Test interactif des règles CLIPS

Nous chargeons le fichier `.clp` dans l'environnement CLIPS, puis proposons un test interactif permettant à l'utilisateur d'entrer un jeu et d'observer les recommandations générées.

In [ ]:
# %%
# Initialisation de l'environnement CLIPS
environment = clips.Environment()
environment.load("regles_steam.clp")

# Fonction de test interactif

def clips_test():
    environment.reset()
    jeu_test = input("Entrez le nom du jeu à tester (ex : HalfLife): ")
    environnement.assert_string(f"(achat {sanitize(jeu_test)})")
    environment.run()

    # Affichage des faits résultants
    for fact in environment.facts():
        print(fact)

# Boucle de test
try:
    clips_test()
except KeyboardInterrupt:
    print("Fin du test interactif.")